# Two-Stage OSR Runner (Colab / VS Code Colab Kernel)

Pure-Python notebook cells (no `!` or `%` magics) to avoid syntax issues in VS Code.

In [1]:
import os, sys, subprocess,torch

print("python:", sys.executable)
print("cwd:", os.getcwd())

try:
    subprocess.run(["nvidia-smi"], check=False)
except FileNotFoundError:
    print("nvidia-smi not found; checking torch CUDA ...")
try:
    print("torch:", torch.__version__)
    print("cuda available:", torch.cuda.is_available())
    if torch.cuda.is_available():
        print("gpu count:", torch.cuda.device_count())
        print("gpu name:", torch.cuda.get_device_name(0))
except Exception as e:
    print("torch check failed:", e)

In [4]:
# Optional: mount Drive if running on Colab
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print('Drive mounted')
except Exception as e:
    print('Drive mount skipped:', e)

In [5]:
from pathlib import Path
import os, subprocess

REPO_URL = 'https://github.com/spinelessknave8/FYP_code.git'
REPO_DIR = Path('/content/FYP-code')

if not REPO_DIR.exists():
    subprocess.check_call(['git', 'clone', REPO_URL, str(REPO_DIR)])

os.chdir(REPO_DIR)
print('repo root:', Path.cwd())
print('has config:', (Path('configs/default.yaml')).exists())
print('has src:', (Path('src')).exists())

In [6]:
import importlib, subprocess, sys

def has(mod):
    try:
        importlib.import_module(mod)
        return True
    except Exception:
        return False

core = [
    ('numpy', 'numpy<2'),
    ('scipy', 'scipy>=1.10'),
    ('PIL', 'pillow>=9.5'),
    ('sklearn', 'scikit-learn>=1.2'),
    ('matplotlib', 'matplotlib>=3.7'),
    ('tqdm', 'tqdm>=4.65'),
    ('yaml', 'pyyaml>=6.0'),
]
missing = [req for mod, req in core if not has(mod)]
if missing:
    print('Installing missing core deps:', missing)
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', *missing])
else:
    print('Core deps already available')

if not has('torch') or not has('torchvision'):
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', 'torch>=2.0', 'torchvision>=0.15'])

import torch, torchvision
print('torch:', torch.__version__, 'torchvision:', torchvision.__version__)

In [7]:
from pathlib import Path

sev = Path("/content/drive/MyDrive/datasets/severstal")
neu = Path("/content/drive/MyDrive/datasets/neu")

print("severstal drive path exists:", sev.exists())
print("neu drive path exists:", neu.exists())

if not sev.exists() or not neu.exists():
    raise RuntimeError("Drive datasets not found at /content/drive/MyDrive/datasets")

In [8]:
# Optional: force GPU in temporary config copy
import yaml
from pathlib import Path
cfg = yaml.safe_load(Path('configs/default.yaml').read_text())
cfg['device'] = 'cuda'
Path('configs/default.colab.yaml').write_text(yaml.safe_dump(cfg, sort_keys=False))
print('wrote configs/default.colab.yaml')

In [ ]:
import time
import yaml
from pathlib import Path
from src.pipelines.notebook_entrypoints import run_two_stage_stage1, run_split_pipeline

t_all = time.time()
print("[0/6] Building Colab config...")

cfg = yaml.safe_load(Path("configs/default.yaml").read_text())
cfg["device"] = "cuda"
cfg["severstal"]["data_root"] = "/content/drive/MyDrive/datasets/severstal"
cfg["severstal"]["train_csv"] = "train.csv"
cfg["severstal"]["images_dir"] = "train_images"
cfg["neu"]["data_root"] = "/content/drive/MyDrive/datasets/neu"

Path("configs/default.colab.yaml").write_text(yaml.safe_dump(cfg, sort_keys=False))
print("  wrote configs/default.colab.yaml")

print("[1/6] Running sanity checks...")
assert Path("/content/drive/MyDrive/datasets/severstal/train.csv").exists(), "Missing Severstal train.csv"
assert Path("/content/drive/MyDrive/datasets/severstal/train_images").exists(), "Missing Severstal train_images/"
assert Path("/content/drive/MyDrive/datasets/neu").exists(), "Missing NEU dataset folder"
print("  sanity checks passed")

print("[2/6] Stage 1: PatchCore training/calibration")
t = time.time()
run_two_stage_stage1("configs/default.colab.yaml")
print(f"  stage 1 done in {time.time() - t:.1f}s")

for i, split in enumerate(["configs/neu_split_a.yaml", "configs/neu_split_b.yaml", "configs/neu_split_c.yaml"], start=3):
    print(f"[{i}/6] Split pipeline: {split}")
    t = time.time()
    run_split_pipeline(split)
    print(f"  {split} done in {time.time() - t:.1f}s")

print(f"[6/6] All done in {time.time() - t_all:.1f}s")

In [ ]:
import json
from pathlib import Path
for split in ['split_a', 'split_b', 'split_c']:
    p = Path('outputs') / split / 'cascade' / 'metrics.json'
    if not p.exists():
        print(split, 'missing metrics')
        continue
    m = json.loads(p.read_text())
    print(split, {
        'tpr_unknown_system': m.get('tpr_unknown_system'),
        'fpr_known_system': m.get('fpr_known_system'),
        'stage1_pass_rate_known': m.get('stage1_pass_rate_known'),
        'stage1_pass_rate_unknown': m.get('stage1_pass_rate_unknown'),
    })